In [3]:
import numpy
from gen2_analysis import factory, diffuse, plotting, effective_areas
%matplotlib inline
import matplotlib.pyplot as plt

In [11]:
# cook up the custom HCR geometry
cos_theta = factory.default_cos_theta_bins
psi_bins = dict(factory.default_psi_bins)
the_geom = 'Sunflower_sparse'
surface = effective_areas.get_fiducial_surface(the_geom, spacing=240, padding=0)
area = surface.azimuth_averaged_area(-1.)/1e6 # what is the footprint size for straight downgoing events
print("Surface Area is {}".format(area))

opts = dict(geometry=the_geom, spacing=240,veto_area=area, veto_threshold=1e5)
kwargs = {
    'cos_theta': cos_theta,
    'psi_bins': psi_bins,
#     'psf_class': None
}
factory.add_configuration('Gen2-HCR', factory.make_options(**opts), **kwargs)

Surface Area is 3.53509545055


In [12]:
# change the factory defaults
factory.set_kwargs(psi_bins={k: [0, numpy.pi] for k in ('tracks', 'cascades', 'radio')})
factory.set_kwargs(cos_theta=[-1,1])

def make_components(aeffs):
    nu, mu = aeffs
    energy_threshold = numpy.inf
    astro = diffuse.DiffuseAstro(nu, livetime=1)
    return dict(astro=astro)

aeffs = factory.get('IceCube')
unshadow_nu, unshadow_mu = aeffs['unshadowed_tracks']
energy_edges = unshadow_nu.get_bin_edges('reco_energy')
energy_centers = unshadow_nu.get_bin_centers('reco_energy')
mask = energy_centers > 1e5

bundle = factory.component_bundle({'IceCube': 0, 'Gen2-InIce': 0, 'Gen2-HCR': 1}, make_components)
components = bundle.get_components()
expectations = components['astro'].expectations(gamma=-2.)
total = numpy.zeros([len(energy_centers)]) # empty array for total numbers
for stream, counts in expectations.iteritems():
    if 'shadow' in stream: # count both the shadowed and unshadowed tracks
        total += counts

total_num = numpy.sum(total[mask])
print("total num above 100 TeV {}".format(total_num))

Warning! Hack! For energy resolution, overriding the requested geometry (Sunflower_sparse) with the standard (Sunflower)
total num above 100 TeV 47.6843672085


In [29]:
# plt.plot(*plotting.stepped_path(energy_edges, total))
# plt.legend()
# plt.loglog()
# plt.ylim(1e-2,1e3)